# Check instances before proceeding

Click on upper right corner

### Image Pytorch 2.0.0 python 3.10 GPU optimised
### Kernel Python 3
### Instance ml g4dn xlarge

* If you stop the training midway.. and attempt to restart the training you'll run into a CUDA OOM error
* Restart the kernel and run all if you land up in the above situation

In [2]:
!nvidia-smi

Wed Nov  1 04:30:37 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:1E.0 Off |                    0 |
| N/A   23C    P0    25W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!free -h --si | awk '/Mem:/ {print $2}'

15G


## Change the kernel and image by Menu bar -> kernel -> change kernel
### Use image pytorch 2.0.0 and python 3.10 GPU optimised  
### Instance ml.g4dn.xlarge

In [4]:
# export 'PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:512'

# Install required libraries

In [5]:
# !pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [8]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops py7zr tensorboardX
!pip install -q -U torch==2.0.1 bitsandbytes==0.40.2


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


# Download the model

In [10]:
import torch, time
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

start = time.time()

In [11]:
# ! pip install flash_attn

In [12]:
# model_name = "ybelkada/falcon-7b-sharded-bf16"
model_name = "tiiuae/falcon-7b" 

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4", # normalised float 4 bit
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
# model = FalconForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False



Loading checkpoint shards: 100%|██████████| 2/2 [01:03<00:00, 31.63s/it]


In [13]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [14]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [15]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

# Download Dataset

In [16]:
from datasets import load_dataset

# Load dataset from the hub
dataset = load_dataset("samsum")

print(f"Train dataset size: {len(dataset['train'])}")
print(f"Test dataset size: {len(dataset['test'])}")

Train dataset size: 14732
Test dataset size: 819


## Convert data to the prompt template for finetuning

In [17]:
from random import randint

# custom instruct prompt start
prompt_template = f"Summarize the chat dialogue:\n{{dialogue}}\n---\nSummary:\n{{summary}}{{eos_token}}"

# template dataset to add prompt to each sample
def template_dataset(sample):
    sample["text"] = prompt_template.format(dialogue=sample["dialogue"],
                                            summary=sample["summary"],
                                            eos_token=tokenizer.eos_token)
    return sample


# apply prompt template per sample
train_dataset = dataset["train"].map(template_dataset, remove_columns=list(dataset["train"].features))

print(train_dataset[randint(0, len(dataset))]["text"])

Summarize the chat dialogue:
Edward: Rachel, I think I'm in ove with Bella..
rachel: Dont say anything else..
Edward: What do you mean??
rachel: Open your fu**ing door.. I'm outside
---
Summary:
Edward thinks he is in love with Bella. Rachel wants Edward to open his door. Rachel is outside. <|endoftext|>


In [18]:
print(train_dataset[2]["text"])

Summarize the chat dialogue:
Tim: Hi, what's up?
Kim: Bad mood tbh, I was going to do lots of stuff but ended up procrastinating
Tim: What did you plan on doing?
Kim: Oh you know, uni stuff and unfucking my room
Kim: Maybe tomorrow I'll move my ass and do everything
Kim: We were going to defrost a fridge so instead of shopping I'll eat some defrosted veggies
Tim: For doing stuff I recommend Pomodoro technique where u use breaks for doing chores
Tim: It really helps
Kim: thanks, maybe I'll do that
Tim: I also like using post-its in kaban style
---
Summary:
Kim may try the pomodoro technique recommended by Tim to get more stuff done.<|endoftext|>


## Set up PEFT parameters 

In [19]:
from peft import LoraConfig, get_peft_model

# Alpha parameter for LoRA scaling: The weight matrix is scaled by lora_alpha/lora_r
lora_alpha = 16
# Dropout probability for LoRA layers
lora_dropout = 0.1
# LoRA attention dimension:  rank of the update matrices : smaler => faster & fewer params to train
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "query_key_value",
        "dense",
        "dense_h_to_4h",
        "dense_4h_to_h",
    ]
)

print_trainable_parameters(model)

trainable params: 0 || all params: 3608744832 || trainable%: 0.0


## Configure the training arguments
We are limiting this to 500 steps.   
You might want to run the training For longer  

In [20]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
# max_steps = 500 # takes ~ 3 hours for 500 steps
max_steps = 150 
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

In [21]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

Detected kernel version 4.14.322, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [22]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [23]:
trainer.train() # ~20 mins

You are using 8-bit optimizers with a version of `bitsandbytes` < 0.41.1. It is recommended to update your version as a major bug has been fixed in 8-bit optimizers.
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,2.033900
20,1.835000
30,1.795600
40,1.703700
50,1.595800
60,1.970000
70,1.820400
80,1.751000
90,1.694200
100,1.578700


TrainOutput(global_step=150, training_loss=1.766760959625244, metrics={'train_runtime': 2023.0745, 'train_samples_per_second': 1.186, 'train_steps_per_second': 0.074, 'total_flos': 1.938644656052736e+16, 'train_loss': 1.766760959625244, 'epoch': 0.16})

In [24]:
trainer.save_model("./results/falcon-7b-finetuned-4bit")

In [25]:
from datetime import timedelta
end = time.time()
print(timedelta(seconds=end-start))

0:35:06.654445
